In [1]:
import os
import psycopg2
import requests
import json
import datetime
from pytz import timezone
import pandas as pd
import requests

In [3]:
rate = requests.get('https://api.coin.z.com/public/v1/ticker?symbol=').json()
df_rate = pd.DataFrame(rate['data'])


In [9]:
host="rdsnske.c7sczatjnphy.ap-northeast-1.rds.amazonaws.com"
dbname="bot"
user="rdsuser"
password="rds%5678"
insert_base = "insert into gmo_rate values('{basetime}','{symbol}','{ask}','{bid}','{high}','{last}','{low}','{volume}');"

In [10]:
# データをDBに登録
with psycopg2.connect(host=host, dbname=dbname, user=user, password=password) as conn:
    with conn.cursor() as cur:
        basetime = datetime.datetime.now()
        basetime = basetime.astimezone(timezone('UTC'))
        basetime = datetime.datetime.strftime(basetime, '%Y-%m-%dT%H:%M:00')
        for d in df_rate.itertuples():
            insert_sql = insert_base.format(basetime=basetime
                                            ,symbol=d.symbol
                                            ,ask=d.ask
                                            ,bid=d.bid
                                            ,high=d.high
                                            ,last=d.last
                                            ,low=d.low
                                            ,volume=d.volume)
            cur.execute(insert_sql)
    conn.commit()

In [11]:
print(insert_sql)

insert into gmo_rate values('2021-11-03T03:46:00','XYM','45.687','45.537','45.85','45.541','43.7','7543850');
